In [1]:
!ls data/

AirPassengers.csv  donations.csv  emails.csv  UNRATE.csv  year_joined.csv


In [2]:
import pandas as pd
from datetime import datetime

In [3]:
emails = pd.read_csv("data/emails.csv")
#https://datatofish.com/strings-to-datetime-pandas/
# emails['week'] = pd.datetime(emails['week'], format="%Y-%m-%d %H:%M:%S")

In [4]:
datetime.strptime('2017-12-04 00:00:40', "%Y-%m-%d %H:%M:%S")

datetime.datetime(2017, 12, 4, 0, 0, 40)

In [5]:
emails['week'] = emails['week'] .apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

You could also ask whether null weeks are reported? That is, do the weeks in
which the member opened 0 emails have a place in the table? This matters when
we want to do time-oriented modeling. In such cases we need to always have the
null weeks present in the data because a 0 week is still a data point.

In [6]:
emails[emails['emailsOpened'] < 1]

,emailsOpened,user,week


There are two possibilities: either nulls are not reported or members always have
at least one email event. Anyone who has worked with email data knows that it’s
difficult to get people to open emails, so the hypothesis that members always
open at least one email per week is quite unlikely. In this case, we can resolve
open at least one email per week is quite unlikely. In this case, we can resolve
this by looking at the history of just one user:


In [7]:
emails[emails['user'] == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04
25465,3.0,998.0,2017-12-11
25466,3.0,998.0,2017-12-18
25467,3.0,998.0,2018-01-01
25468,3.0,998.0,2018-01-08
25469,2.0,998.0,2018-01-15
25470,3.0,998.0,2018-01-22
25471,2.0,998.0,2018-01-29
25472,3.0,998.0,2018-02-05
25473,3.0,998.0,2018-02-12


We can see that some weeks are missing. There aren’t any December 2017 email
events after December 18, 2017.


In [8]:
emails.dtypes

emailsOpened           float64
user                   float64
week            datetime64[ns]
dtype: object

Lets see how to verify this mathematically...

In [9]:
week_entries = emails[emails['user'] == 998]['week']
min(week_entries)

Timestamp('2017-12-04 00:00:00')

In [10]:
def get_number_of_days(week_entries):
    return (max(week_entries) - min(week_entries)).days/7

In [11]:
get_number_of_days(week_entries)

25.0

In [12]:
week_entries.shape

(24,)

Lets see whats the difference or missing weeks for a set of users...

In [13]:
for user, grp in emails.groupby(['user']):
    if user in [1.0,3.0,34.0]: #Random sample ids
        print(f"User id: {user} Expected : {get_number_of_days(grp['week'])} Actual: {grp.shape[0]}")

User id: 1.0 Expected : 152.0 Actual: 139
User id: 3.0 Expected : 7.0 Actual: 8
User id: 34.0 Expected : 149.0 Actual: 136


### Pandas Multi Index

A MultiIndex, also known as a multi-level index or hierarchical index, allows you to have multiple columns acting as a row identifier, while having each index column related to another through a parent/child relationship.



In [16]:
emails.index

RangeIndex(start=0, stop=25488, step=1)

It’s a lot easier to fill in all missing weeks for all members by exploiting Pandas’
indexing functionality, rather than writing our own solution. We can generate a
MultiIndex for a Pandas data frame, which will create all combinations of
weeks and members—that is, a Cartesian product:


In [19]:
complete_idx = pd.MultiIndex.from_product((set(emails.week),set(emails.user)))
complete_idx

MultiIndex([('2017-11-20',   1.0),
            ('2017-11-20',   3.0),
            ('2017-11-20',   5.0),
            ('2017-11-20',   6.0),
            ('2017-11-20',   9.0),
            ('2017-11-20',  10.0),
            ('2017-11-20',  14.0),
            ('2017-11-20',  16.0),
            ('2017-11-20',  20.0),
            ('2017-11-20',  21.0),
            ...
            ('2016-01-11', 973.0),
            ('2016-01-11', 977.0),
            ('2016-01-11', 982.0),
            ('2016-01-11', 984.0),
            ('2016-01-11', 987.0),
            ('2016-01-11', 991.0),
            ('2016-01-11', 992.0),
            ('2016-01-11', 993.0),
            ('2016-01-11', 995.0),
            ('2016-01-11', 998.0)],
           length=93247)

In [27]:
# First set the index with week and user id and fill the missing value in emailOpened to 0 and then reset the index to original iteger based!
all_emails = emails.set_index(['week', 'user']).reindex(complete_idx, fill_value=0).reset_index()
all_emails.columns = ["week", "member", "emailsOpened"]

In [25]:
all_emails

,week,member,emailsOpened
0,2017-11-20,1.0,3.0
1,2017-11-20,3.0,0.0
2,2017-11-20,5.0,1.0
3,2017-11-20,6.0,0.0
4,2017-11-20,9.0,3.0
...,...,...,...
93242,2016-01-11,991.0,0.0
93243,2016-01-11,992.0,0.0
93244,2016-01-11,993.0,0.0
93245,2016-01-11,995.0,0.0


What we have done? re-orderd the series baed on date

In [26]:
all_emails[all_emails["member"]==998].sort_values("week")

,week,member,emailsOpened
36651,2015-02-09,998.0,0.0
52821,2015-02-16,998.0,0.0
74381,2015-02-23,998.0,0.0
86239,2015-03-02,998.0,0.0
2694,2015-03-09,998.0,0.0
...,...,...,...
29105,2018-04-30,998.0,3.0
64140,2018-05-07,998.0,3.0
14013,2018-05-14,998.0,3.0
59828,2018-05-21,998.0,3.0
